# Diebold Mariano Test

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from itertools import combinations

# Define a function for the Diebold–Mariano (DM) test using squared error loss.
def dm_test(e2, e1, h=1):
    """
    Perform the Diebold–Mariano test comparing two forecast error series using squared error loss.
    
    Arguments:
      e1, e2 : np.array
         Forecast error series for model 1 and model 2 (e.g. actual - forecast)
      h : int, optional
         Forecast horizon (default is 1). For h=1, the variance is the usual sample variance.
    
    Returns:
      dm_stat : float
         The DM test statistic
      p_value : float
         The two–sided p-value.
    """
    # compute the loss differential series d_t = e1^2 - e2^2
    d = e1**2 - e2**2
    T = len(d)
    d_bar = np.mean(d)
    if T <= 1:
        return np.nan, np.nan
    # For h=1, simply use the sample variance.
    if h == 1:
        V_d = np.var(d, ddof=1) / T
    else:
        m = h - 1
        gamma0 = np.sum((d - d_bar)**2) / T
        gamma_sum = 0
        for j in range(1, m+1):
            weight = 1 - j/m
            gamma = np.sum((d[j:] - d_bar) * (d[:-j] - d_bar)) / T
            gamma_sum += weight * gamma
        V_d = (gamma0 + 2 * gamma_sum) / T
    dm_stat = d_bar / np.sqrt(V_d)
    p_value = 2 * (1 - norm.cdf(abs(dm_stat)))
    return dm_stat, p_value

# --- Assume your three DataFrames are already loaded ---
# OLS has columns: "Year", "y_actual", "y_pred"
# NN has columns: "Year", "Actual", "Predicted"
# RF has columns: "Year", "Predicted_y", "Actual_y"
# RW has columns: "Year", "actual", "predicted"
df_ols = pd.read_csv("dm_test_results_OLS3.csv")        # For the OLS forecasts
df_nn = pd.read_csv("dm_test_results_NN.csv")    # Alternative Forecast 1
df_rf = pd.read_csv("dm_test_results_RF.csv")    # Alternative Forecast 2
df_rw = pd.read_csv("dm_test_randomwalk.csv")

# Remove observations from 2016
df_ols = df_ols[df_ols['Year'] != 2016].reset_index(drop=True)
df_rf = df_rf[df_rf['Year'] != 2016].reset_index(drop=True)
df_rw = df_rw[df_rw['year'] != 2016].reset_index(drop=True)
df_nn = df_nn.reset_index(drop=True)

df_ols.rename(columns={'y_actual': 'Actual',
                       'y_pred':   'OLS_pred'}, inplace=True)
df_nn.rename(columns={'Predicted': 'NN_pred'}, inplace=True)
# For the RF data we want the RF forecast as “RF_pred”. We assume the actual value in RF is identical to OLS.
df_rf.rename(columns={'Predicted_y': 'RF_pred',
                      'Actual_y':    'Actual_RF'}, inplace=True)
df_rw.rename(columns={'predicted': 'RW_pred',
                      'year': 'Year'}, inplace=True)

rf_preds = df_rf[['RF_pred']].values  # make sure this is a 2D array for StandardScaler
rw_preds = df_rw[['RW_pred']].values  # make sure this is a 2D array for StandardScaler
# Sort all DataFrames by Year and Actual
df_ols.sort_values(by=['Actual','Year'], inplace=True)
df_nn.sort_values(by=['Actual','Year'], inplace=True)
df_rf.sort_values(by=['Actual_RF','Year'], inplace=True)
df_rw.sort_values(by=['actual','Year'], inplace=True)

df_ols.reset_index(drop=True, inplace=True)
df_nn.reset_index(drop=True, inplace=True)
df_rf.reset_index(drop=True, inplace=True)
df_rw.reset_index(drop=True, inplace=True)

df_merged = pd.concat([
    df_ols[['Year', 'Actual', 'OLS_pred']],
    df_nn[['NN_pred']],
    df_rf[['RF_pred']],
    df_rw[['RW_pred']]
], axis=1)

# In case the "Year" columns are duplicated, remove duplicates
df_merged = df_merged.loc[:, ~df_merged.columns.duplicated()]

# Define model names
models = ['OLS', 'NN', 'RF', 'RW']

# Prepare a dictionary to store results
results_dict = {}

# Iterate over each year to create 4x4 tables
for yr, grp in df_merged.groupby('Year'):
    errors = {
        'OLS': grp['Actual'] - grp['OLS_pred'],
        'NN': grp['Actual'] - grp['NN_pred'],
        'RF': grp['Actual'] - grp['RF_pred'],
        'RW': grp['Actual'] - grp['RW_pred']
    }
    
    # Create an empty DataFrame for the results
    dm_matrix = pd.DataFrame(index=models, columns=models)
    
    # Compute DM statistics for each model pair
    for m1, m2 in combinations(models, 2):
        dm_stat, p_value = dm_test(errors[m1].values, errors[m2].values, h=1)
        dm_matrix.loc[m1, m2] = f"{dm_stat:.3f} (p={p_value:.3f})"
        dm_matrix.loc[m2, m1] = f"{dm_stat:.3f} (p={p_value:.3f})"
    
    # Store in dictionary
    results_dict[yr] = dm_matrix

# Print results for each year
for year, table in results_dict.items():
    print(f"\nDM Test Results for Year {year}")
    print(table.fillna("-"))


DM Test Results for Year 2017
                  OLS                NN                RF               RW
OLS                 -  -2.083 (p=0.037)  -1.309 (p=0.190)  8.479 (p=0.000)
NN   -2.083 (p=0.037)                 -   1.573 (p=0.116)  7.973 (p=0.000)
RF   -1.309 (p=0.190)   1.573 (p=0.116)                 -  8.142 (p=0.000)
RW    8.479 (p=0.000)   7.973 (p=0.000)   8.142 (p=0.000)                -

DM Test Results for Year 2018
                  OLS                NN                RF               RW
OLS                 -  -1.426 (p=0.154)  -2.731 (p=0.006)  9.824 (p=0.000)
NN   -1.426 (p=0.154)                 -  -1.844 (p=0.065)  9.717 (p=0.000)
RF   -2.731 (p=0.006)  -1.844 (p=0.065)                 -  9.965 (p=0.000)
RW    9.824 (p=0.000)   9.717 (p=0.000)   9.965 (p=0.000)                -

DM Test Results for Year 2019
                  OLS                NN                RF                RW
OLS                 -  -1.859 (p=0.063)  -2.473 (p=0.013)  12.743 (p=0.000)
NN   

In [48]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from itertools import combinations

# Define a function for the Diebold–Mariano (DM) test using squared error loss.
def dm_test(e2, e1, h=1):
    d = e1**2 - e2**2
    T = len(d)
    d_bar = np.mean(d)
    if T <= 1:
        return np.nan, np.nan
    V_d = np.var(d, ddof=1) / T if h == 1 else (np.sum((d - d_bar)**2) / T + 2 * sum(
        (1 - j/(h-1)) * np.sum((d[j:] - d_bar) * (d[:-j] - d_bar)) / T for j in range(1, h))) / T
    dm_stat = d_bar / np.sqrt(V_d)
    p_value = 2 * (1 - norm.cdf(abs(dm_stat)))
    return dm_stat, p_value

# Define model names
models = ['OLS', 'NN', 'RF', 'RW']

# Prepare a dictionary to store results
results_dict = {}

# Iterate over each year to create 4x4 tables
for yr, grp in df_merged.groupby('Year'):
    errors = {
        'OLS': grp['Actual'] - grp['OLS_pred'],
        'NN': grp['Actual'] - grp['NN_pred'],
        'RF': grp['Actual'] - grp['RF_pred'],
        'RW': grp['Actual'] - grp['RW_pred']
    }
    
    dm_matrix = pd.DataFrame(index=models, columns=models)
    
    for m1, m2 in combinations(models, 2):
        dm_stat, p_value = dm_test(errors[m1].values, errors[m2].values, h=1)
        dm_matrix.loc[m1, m2] = f"{dm_stat:.3f} (p={p_value:.3f})"
        dm_matrix.loc[m2, m1] = f"{dm_stat:.3f} (p={p_value:.3f})"
    
    results_dict[yr] = dm_matrix

# Compute DM test across all years
all_errors = {model: df_merged['Actual'] - df_merged[f'{model}_pred'] for model in models}
dm_matrix_all = pd.DataFrame(index=models, columns=models)
for m1, m2 in combinations(models, 2):
    dm_stat, p_value = dm_test(all_errors[m1].values, all_errors[m2].values, h=1)
    dm_matrix_all.loc[m1, m2] = f"{dm_stat:.3f} (p={p_value:.3f})"
    dm_matrix_all.loc[m2, m1] = f"{dm_stat:.3f} (p={p_value:.3f})"

# Print results for each year
for year, table in results_dict.items():
    print(f"\nDM Test Results for Year {year}")
    print(table.fillna("-"))

# Print results for all years
print("\nDM Test Results Across All Years")
print(dm_matrix_all.fillna("-"))



DM Test Results for Year 2017
                  OLS                NN                RF               RW
OLS                 -  -2.083 (p=0.037)  -1.309 (p=0.190)  8.479 (p=0.000)
NN   -2.083 (p=0.037)                 -   1.573 (p=0.116)  7.973 (p=0.000)
RF   -1.309 (p=0.190)   1.573 (p=0.116)                 -  8.142 (p=0.000)
RW    8.479 (p=0.000)   7.973 (p=0.000)   8.142 (p=0.000)                -

DM Test Results for Year 2018
                  OLS                NN                RF               RW
OLS                 -  -1.426 (p=0.154)  -2.731 (p=0.006)  9.824 (p=0.000)
NN   -1.426 (p=0.154)                 -  -1.844 (p=0.065)  9.717 (p=0.000)
RF   -2.731 (p=0.006)  -1.844 (p=0.065)                 -  9.965 (p=0.000)
RW    9.824 (p=0.000)   9.717 (p=0.000)   9.965 (p=0.000)                -

DM Test Results for Year 2019
                  OLS                NN                RF                RW
OLS                 -  -1.859 (p=0.063)  -2.473 (p=0.013)  12.743 (p=0.000)
NN   